<a href="https://colab.research.google.com/github/ahigerd/scheme-lessons/blob/master/Calysto_Landon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install --upgrade calysto-scheme --user
!python3 -m calysto_scheme install --user

     |████████████████████████████████| 317kB 2.7MB/s 
     |████████████████████████████████| 215kB 8.7MB/s 
  Created wheel for yasi: filename=yasi-2.1.2-cp36-none-any.whl size=20687 sha256=289fcfa23ab454236cd6296157d1b372de71f27b883800c5084da9ecb2806cbc
  Stored in directory: /root/.cache/pip/wheels/61/fd/0b/8cb55b23a37b324c6e713804eaf594f9abb375d1eaf6dd4931
Successfully built yasi
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
[InstallKernelSpec] Removing existing kernelspec in /root/.local/share/jupyter/kernels/calysto_scheme
[InstallKernelSpec] Installed kernelspec calysto_scheme in /root/.local/share/jupyter/kernels/calysto_scheme


In [43]:
(define string-lower (python-eval "str.lower"))
(define (string-ci=? lhs rhs) 
  (string=?
    (string-lower lhs)
    (string-lower rhs)
  )
)

 (define (lookup-word w l)
    (if (pair? l)
        (if (string-ci=? w (caar l))
            (car l)
            (lookup-word w (cdr l)))
        '()))
  
  (define (add-word w f l)
    (define (add-follow d ac)
      (if (pair? d)
          (if (string? (car d))
              (add-follow (cdr d) (list (car d)))
              (if (string-ci=? f (caar d))
                  (append ac (list (list f (+ (cadar d) 1))) (cdr d))
                  (add-follow (cdr d) (append ac (list (car d))))))
          (append ac (list (list f 1)))))
    (define (aw-helper l accum)
      (if (pair? l)
          (if (string-ci=? w (caar l))
              (append accum (list (add-follow (car l) '())) (cdr l))
              (aw-helper (cdr l) (append accum (list (car l)))))
          '()))
    (if (eq? l '())
        (list (list w (list f 1)))
        (if (eq? (lookup-word w l) '())
            (append l (list (list w (list f 1))))
            (aw-helper l '()))))
  
  (define (cleanup word)
    (define cl (string->list word))
    (define (cu-helper l)
      (if (pair? l)
          (if (char-alphabetic? (car l))
              (cons (car l) (cleanup (cdr l))))
          '()))
    (list->string (cu-helper cl)))
  
  (define (char-word? ch)
    (or (char-alphabetic? ch)
        (char=? #\' ch)))
  
  (define (first-word l)
    (if (pair? l)
        (if (char-word? (car l))
            (cons (car l) (first-word (cdr l)))
            '())
        '()))
  
  (define (strip-junk l)
    (if (pair? l)
        (if (and (not (char-word? (car l)))
                 (not (char=? (car l) #\.)))
            (strip-junk (cdr l))
            l)
        '()))
  
  (define (strip-first l)
    (if (pair? l)
        (if (char-word? (car l))
            (strip-first (cdr l))
            (list->string (strip-junk l)))
        ""))
  
  (define (wordify s)
    (define cl (string->list s))
    (if (pair? cl)
        (if (char=? (car cl) #\.)
            (cons "." (wordify (list->string (strip-junk (cdr cl)))))
            (cons (list->string (first-word cl)) (wordify (strip-first cl))))
        '()))
  
  (define (add-sentence s l)
    (define wl (cons "." (wordify s)))
    (define (as-helper sl l)
      (if (pair? sl)
          (if (pair? (cdr sl))
              (as-helper (cdr sl) (add-word (car sl) (cadr sl) l))
              (if (not (string=? (car sl) "."))
                  (add-word (car sl) "." l)
                  l))
          l))
    (if (pair? (cdr wl))
        (as-helper wl l)
        l))
  
  (define (assemble-sentence l)
    (if (pair? l)
        (if (pair? (cdr l))
            (string-append (car l) " " (assemble-sentence (cdr l)))
            (car l))
        ""))
  
  (define (assoc-sum l)
    (apply + (map cadr l)))
  
  (define (assoc-ref l n)
    (if (and (pair? l) (>= n 0))
        (if (> (cadar l) n)
            (car l)
            (assoc-ref (cdr l) (- n (cadar l))))
        '()))
  
  (define (unstack l)
    (define (us-helper l n)
      (define r (assoc-ref l n))
      (if (pair? r)
          (cons (car r) (us-helper l (+ n 1)))
          '()))
    (us-helper l 0))
  
  ; ASSERTION: (assoc-ref l n) == (list-ref (unstack l) n)
  
  (define (generate-sentence l)
    (define (gs-helper i)
      (define wl (cdr (lookup-word i l)))
      (define rn (random (assoc-sum wl)))
      (define nw (assoc-ref wl rn))
      (if (equal? "." (car nw))
          '()
          (cons (car nw) (gs-helper (car nw)))))
    (if (not (pair? l))
        "** No dictionary present **"
        (string-append (assemble-sentence (gs-helper ".")) ".")))
  
  (define (main dict)
    (define rl (read))
    (define new-dict 
      (if (not (= 0 (string-length rl)))
          (add-sentence rl dict)
          #f))
    (if (not (eq? new-dict #f))
        (begin
          (display "<Landon> ")
          (display (generate-sentence new-dict))
          (newline)
          (display "<User> ")
          (main new-dict))))
  
  (display "<User> ")
  (main '())


<User> I know a song that gets on everybody's nerves.
<Landon> I know a song that gets on everybody's nerves.
<User> I know I can make a terrible example.
<Landon> I can make a terrible example.
<User> You are a rather silly chatbot.
<Landon> You are a terrible example.
<User> 


#f